# Temperature data per country

Monthly temperature data for countries comes from the [World Bank](https://climateknowledgeportal.worldbank.org/download-data).

In [1]:
import numpy as np
import pickle
import pandas as pd
import os
from sklearn import preprocessing as pre

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [20]:
temp = pd.read_csv('utils/data/temp.csv', sep=', ', usecols=range(4), dtype=object)
temp.tail()

,Temperature - (Celsius),Year,Statistics,Country
61147,18.7555,2016,Aug Average,Zimbabwe
61148,22.9919,2016,Sep Average,Zimbabwe
61149,25.8664,2016,Oct Average,Zimbabwe
61150,25.8515,2016,Nov Average,Zimbabwe
61151,25.0693,2016,Dec Average,Zimbabwe


In [24]:
countries = []
c = temp.Country.unique()

for country in c:
    countries.append(country.strip())

In [26]:
temp_dict = {}
t_dict = {}
t_dict_std = {}

for country in countries:
    temp_dict[country] = temp[temp.Country == country]
    temp_dict[country].drop(columns=['Statistics', 'Country'], inplace=True)
    t_dict[country] = temp_dict[country].astype(float).groupby('Year')['Temperature - (Celsius)'].mean()
    t_dict[country].rename('Temperature', inplace=True)
    t_dict_std[country] = pd.Series(index=t_dict[country].index, data=pre.scale(list(t_dict[country])))
    t_dict_std[country].rename('Temperature', inplace=True)

In [27]:
# check
t_dict_std['Albania']

Year
1991.0   -2.040162
1992.0   -1.035685
1993.0   -0.974972
1994.0    0.802400
1995.0   -1.452880
1996.0   -1.805582
1997.0   -1.327218
1998.0   -0.262997
1999.0    0.279129
2000.0    0.788818
2001.0    0.285842
2002.0    0.437547
2003.0    0.061019
2004.0   -0.424538
2005.0   -1.337297
2006.0   -0.636711
2007.0    0.702137
2008.0    0.943248
2009.0    0.627456
2010.0    0.542144
2011.0    0.119702
2012.0    1.062410
2013.0    1.224875
2014.0    1.140745
2015.0    1.104895
2016.0    1.175673
Name: Temperature, dtype: float64

In [28]:
# better save this precious data
t = open('utils/data/temp.pkl', 'wb')
pickle.dump(t_dict_std, t)
t.close()